<a href="https://colab.research.google.com/github/Bitang-Melyen-Tanulok/Csip_Csip/blob/main/Marci_BirdCLEF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!cd /content/drive/MyDrive/DeepLearning

!kaggle competitions download -c birdclef-2024

 19% 4.12G/21.8G [01:05<04:39, 68.0MB/s]
User cancelled operation
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 79, in main
    if out is not None:
KeyboardInterrupt
^C


In [31]:
%cd /content/drive/MyDrive/DeepLearning
!kaggle competitions download -c birdclef-2024

/content/drive/.shortcut-targets-by-id/14L9x4h4L96yptiH9WIzuLnI8-Atjz2zY/DeepLearning
100% 21.8G/21.8G [05:35<00:00, 67.8MB/s]
100% 21.8G/21.8G [05:35<00:00, 69.7MB/s]


In [34]:
path = '/content/drive/MyDrive/DeepLearning'

import zipfile
with zipfile.ZipFile(path+'/birdclef-2024.zip', 'r') as zip_ref:
    zip_ref.extractall(path)